In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import rasterio
import rioxarray
import numpy as np
import matplotlib.pyplot as plt
import gdown
import geopandas as gpd

## Download all data using `gdown` and organize into folders

In [ ]:
#Download KID-13 DEMs
url = 'https://drive.google.com/drive/u/1/folders/1u6UxBlPMZmzgvfbA5-kOsVX_mJ1J55kL'
k13_dem_folder='../data/dems/k13'
# gdown.download_folder(url, output=k13_dem_folder, quiet=False, use_cookies=False)

In [ ]:
#Download MEL-14 DEMs
url = 'https://drive.google.com/drive/u/1/folders/1WkPjWOsFvu3nW9gT4NtuXNg3xSD6OF9p'
m14_dem_folder='../data/dems/m14'
# gdown.download_folder(url, output=m14_dem_folder, quiet=False, use_cookies=False)

In [ ]:
#Download masks for clipping
url = 'https://drive.google.com/drive/u/1/folders/14jvvzh3dSNO140nXurEaMokyDvr6KidB'
k13_mask_folder='../data/vectors/kid_mask'
# gdown.download_folder(url, output=k13_mask_folder, quiet=False, use_cookies=False)

url = 'https://drive.google.com/drive/u/1/folders/1jSSD9YJ6JzxyZDanQrNMIsBVVn6ZGubF'
m14_mask_folder='../data/vectors/mel_mask'
# gdown.download_folder(url, output=m14_mask_folder, quiet=False, use_cookies=False)

In [ ]:
k13_wsyr1_dem_files = sorted(Path(k13_dem_folder).glob('wsyr1/*.tif'))
k13_wsyr1_dem_files = [i.as_posix() for i in k13_wsyr1_dem_files]

k13_dsyr1_dem_files = sorted(Path(k13_dem_folder).glob('dsyr1/*.tif'))
k13_dsyr1_dem_files = [i.as_posix() for i in k13_dsyr1_dem_files]

k13_wsyr2_dem_files = sorted(Path(k13_dem_folder).glob('wsyr2/*.tif'))
k13_wsyr2_dem_files = [i.as_posix() for i in k13_wsyr2_dem_files]

m14_wsyr1_dem_files = sorted(Path(m14_dem_folder).glob('wsyr1/*.tif'))
m14_wsyr1_dem_files = [i.as_posix() for i in m14_wsyr1_dem_files]

m14_dsyr1_dem_files = sorted(Path(m14_dem_folder).glob('dsyr1/*.tif'))
m14_dsyr1_dem_files = [i.as_posix() for i in m14_dsyr1_dem_files]

m14_wsyr2_dem_files = sorted(Path(m14_dem_folder).glob('wsyr2/*.tif'))
m14_wsyr2_dem_files = [i.as_posix() for i in m14_wsyr2_dem_files]

In [ ]:
k13_wsyr1_dem_files, k13_dsyr1_dem_files, k13_wsyr2_dem_files, m14_wsyr1_dem_files, m14_dsyr1_dem_files, m14_wsyr2_dem_files

(['../data/dems/k13/wsyr1/K13_lidar_20201109_cc.tif',
  '../data/dems/k13/wsyr1/K13_lidar_20210513_cc.tif',
  '../data/dems/k13/wsyr1/K13_sfm_20201109_cc.tif',
  '../data/dems/k13/wsyr1/K13_sfm_20210208_cc.tif',
  '../data/dems/k13/wsyr1/K13_sfm_20210406_cc.tif',
  '../data/dems/k13/wsyr1/K13_sfm_20210513_cc.tif'],
 ['../data/dems/k13/dsyr1/K13_lidar_20210604_cc.tif',
  '../data/dems/k13/dsyr1/K13_lidar_20210913_cc.tif',
  '../data/dems/k13/dsyr1/K13_sfm_20210604_cc.tif',
  '../data/dems/k13/dsyr1/K13_sfm_20210819_cc.tif',
  '../data/dems/k13/dsyr1/K13_sfm_20210913_cc.tif'],
 ['../data/dems/k13/wsyr2/K13_lidar_20211007_cc.tif',
  '../data/dems/k13/wsyr2/K13_lidar_20220531_cc.tif',
  '../data/dems/k13/wsyr2/K13_sfm_20211007_cc.tif',
  '../data/dems/k13/wsyr2/K13_sfm_20220208_cc.tif',
  '../data/dems/k13/wsyr2/K13_sfm_20220503_cc.tif',
  '../data/dems/k13/wsyr2/K13_sfm_20220531_cc.tif'],
 ['../data/dems/m14/wsyr1/M14_lidar_20201203_cc.tif',
  '../data/dems/m14/wsyr1/M14_sfm_20201203_cc.t

## Define CRS for data using CRS from masks

In [ ]:
k13_mask_path = '../data/vectors/kid_mask/K13_mask.shp'
m14_mask_path = '../data/vectors/mel_mask/MEL14_Mask.shp'

# grab shapefile crs to write to rasters
k13_crs = gpd.read_file(k13_mask_path).crs
k13_mask_gdf = gpd.read_file(k13_mask_path)

m14_crs = gpd.read_file(m14_mask_path).crs
m14_mask_gdf = gpd.read_file(m14_mask_path)

## Clip and reproject KID-13 data

In [ ]:
#Wet season year 1
k13_wsyr1 = dict()
k13_wsyr1_clip = dict()
for i in range(len(k13_wsyr1_dem_files)):
    k13_wsyr1[i] = rioxarray.open_rasterio(k13_wsyr1_dem_files[i]).squeeze()
    k13_wsyr1[i].rio.write_crs(k13_crs, inplace=True)
    k13_wsyr1_clip[i] = k13_wsyr1[i].rio.clip(k13_mask_gdf.geometry)
    k13_wsyr1_clip[i].rio.write_nodata(-9999, encoded=True, inplace=True)

k13_wsyr1_reproj = dict()
for x in range(len(k13_wsyr1_dem_files)-1):
    k13_wsyr1_reproj[x] = k13_wsyr1_clip[x+1].rio.reproject_match(k13_wsyr1_clip[0], resampling = 3)

#Dry season year 1
k13_dsyr1 = dict()
k13_dsyr1_clip = dict()
for i in range(len(k13_dsyr1_dem_files)):
    k13_dsyr1[i] = rioxarray.open_rasterio(k13_dsyr1_dem_files[i]).squeeze()
    k13_dsyr1[i].rio.write_crs(k13_crs, inplace=True)
    k13_dsyr1_clip[i] = k13_dsyr1[i].rio.clip(k13_mask_gdf.geometry)
    k13_dsyr1_clip[i].rio.write_nodata(-9999, encoded=True, inplace=True)

k13_dsyr1_reproj = dict()
for x in range(len(k13_dsyr1_dem_files)-1):
    k13_dsyr1_reproj[x] = k13_dsyr1_clip[x+1].rio.reproject_match(k13_dsyr1_clip[0], resampling = 3)

#Wet season year 2
k13_wsyr2 = dict()
k13_wsyr2_clip = dict()
for i in range(len(k13_wsyr2_dem_files)):
    k13_wsyr2[i] = rioxarray.open_rasterio(k13_wsyr2_dem_files[i]).squeeze()
    k13_wsyr2[i].rio.write_crs(k13_crs, inplace=True)
    k13_wsyr2_clip[i] = k13_wsyr2[i].rio.clip(k13_mask_gdf.geometry)
    k13_wsyr2_clip[i].rio.write_nodata(-9999, encoded=True, inplace=True)

k13_wsyr2_reproj = dict()
for x in range(len(k13_wsyr2_dem_files)-1):
    k13_wsyr2_reproj[x] = k13_wsyr2_clip[x+1].rio.reproject_match(k13_wsyr2_clip[0], resampling = 3)

## Clip and reproject MEL-14 data

In [10]:
#Wet season year 1
dates = []
m14_wsyr1 = dict()
m14_wsyr1_clip = dict()
for i in range(len(m14_wsyr1_dem_files)):
    m14_wsyr1[i] = rioxarray.open_rasterio(m14_wsyr1_dem_files[i]).squeeze()
    m14_wsyr1[i].rio.write_crs(m14_crs, inplace=True)
    m14_wsyr1_clip[i] = m14_wsyr1[i].rio.clip(m14_mask_gdf.geometry)
    m14_wsyr1_clip[i].rio.write_nodata(-9999, encoded=True, inplace=True)

m14_wsyr1_reproj = dict()
for x in range(len(m14_wsyr1_dem_files)-1):
    m14_wsyr1_reproj[x] = m14_wsyr1_clip[x+1].rio.reproject_match(m14_wsyr1_clip[0], resampling = 3)

#Dry season year 1
dates = []
m14_dsyr1 = dict()
m14_dsyr1_clip = dict()
for i in range(len(m14_dsyr1_dem_files)):
    m14_dsyr1[i] = rioxarray.open_rasterio(m14_dsyr1_dem_files[i]).squeeze()
    m14_dsyr1[i].rio.write_crs(m14_crs, inplace=True)
    m14_dsyr1_clip[i] = m14_dsyr1[i].rio.clip(m14_mask_gdf.geometry)
    m14_dsyr1_clip[i].rio.write_nodata(-9999, encoded=True, inplace=True)

m14_dsyr1_reproj = dict()
for x in range(len(m14_dsyr1_dem_files)-1):
    m14_dsyr1_reproj[x] = m14_dsyr1_clip[x+1].rio.reproject_match(m14_dsyr1_clip[0], resampling = 3)

#Wet season year 2
dates = []
m14_wsyr2 = dict()
m14_wsyr2_clip = dict()
for i in range(len(m14_wsyr2_dem_files)):
    m14_wsyr2[i] = rioxarray.open_rasterio(m14_wsyr2_dem_files[i]).squeeze()
    m14_wsyr2[i].rio.write_crs(m14_crs, inplace=True)
    m14_wsyr2_clip[i] = m14_wsyr2[i].rio.clip(m14_mask_gdf.geometry)
    m14_wsyr2_clip[i].rio.write_nodata(-9999, encoded=True, inplace=True)

m14_wsyr2_reproj = dict()
for x in range(len(m14_wsyr2_dem_files)-1):
    m14_wsyr2_reproj[x] = m14_wsyr2_clip[x+1].rio.reproject_match(m14_wsyr2_clip[0], resampling = 3)

In [ ]:
#clipping using rioxarray
# Wet season, year 1; LiDAR
K13_20201109_LiDAR_clipped = K13_20201109_LiDAR_full.rio.clip(mask_gdf.geometry)
K13_20210513_LiDAR_clipped = K13_20210513_LiDAR_full.rio.clip(mask_gdf.geometry)
K13_20210513_LiDAR_reproj = K13_20210513_LiDAR_clipped.rio.reproject_match(K13_20201109_LiDAR_clipped, resampling=3)
K13_20201109_LiDAR_clipped.rio.to_raster('20201109_KID13_LiDAR.tif')
K13_20210513_LiDAR_reproj.rio.to_raster('20210513_KID13_LiDAR.tif')

# Wet season, year 1; sfm
K13_20201109_sfm_clipped = K13_20201109_sfm_full.rio.clip(mask_gdf.geometry)
K13_20210208_sfm_clipped = K13_20210208_sfm_full.rio.clip(mask_gdf.geometry)
K13_20210406_sfm_clipped = K13_20210406_sfm_full.rio.clip(mask_gdf.geometry)
K13_20210513_sfm_clipped = K13_20210513_sfm_full.rio.clip(mask_gdf.geometry)
K13_20210208_sfm_reproj = K13_20210208_sfm_clipped.rio.reproject_match(K13_20201109_sfm_clipped, resampling=3)
K13_20210406_sfm_reproj = K13_20210406_sfm_clipped.rio.reproject_match(K13_20201109_sfm_clipped, resampling=3)
K13_20210513_sfm_reproj = K13_20210513_sfm_clipped.rio.reproject_match(K13_20201109_sfm_clipped, resampling=3)
K13_20201109_sfm_clipped.rio.to_raster('20201109_KID13_sfm.tif')
K13_20210208_sfm_reproj.rio.to_raster('20210208_KID13_sfm.tif')
K13_20210406_sfm_reproj.rio.to_raster('20210406_KID13_sfm.tif')
K13_20210513_sfm_reproj.rio.to_raster('20210513_KID13_sfm.tif')

# Dry season, year 1; LiDAR
K13_20210604_LiDAR_clipped = K13_20210604_LiDAR_full.rio.clip(mask_gdf.geometry)
K13_20210913_LiDAR_clipped = K13_20210913_LiDAR_full.rio.clip(mask_gdf.geometry)
K13_20210913_LiDAR_reproj = K13_20210913_LiDAR_clipped.rio.reproject_match(K13_20210604_LiDAR_clipped, resampling=3)
K13_20210604_LiDAR_clipped.rio.to_raster('20210604_KID13_LiDAR.tif')
K13_20210913_LiDAR_reproj.rio.to_raster('20210913_KID13_LiDAR.tif')

# Dry season, year 1; sfm
K13_20210604_sfm_clipped = K13_20210604_sfm_full.rio.clip(mask_gdf.geometry)
K13_20210819_sfm_clipped = K13_20210819_sfm_full.rio.clip(mask_gdf.geometry)
K13_20210913_sfm_clipped = K13_20210913_sfm_full.rio.clip(mask_gdf.geometry)
K13_20210819_sfm_reproj = K13_20210819_sfm_clipped.rio.reproject_match(K13_20210604_sfm_clipped, resampling=3)
K13_20210913_sfm_reproj = K13_20210913_sfm_clipped.rio.reproject_match(K13_20210604_sfm_clipped, resampling=3)
K13_20210604_sfm_clipped.rio.to_raster('20210604_KID13_sfm.tif')
K13_20210819_sfm_reproj.rio.to_raster('20210819_KID13_sfm.tif')
K13_20210913_sfm_reproj.rio.to_raster('20210913_KID13_sfm.tif')

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(8,4))

m0 = ax[0].imshow(K13_20201109_LiDAR_clipped, cmap = 'viridis', vmin=501, vmax=506)
ax[0].set_title('KID-13 2020-11-09\nLiDAR')
fig.colorbar(m0, label='Elevation (m)', extend='both', ax=ax[0])
ax[0].set_facecolor('k')
ax[0].set_xticks(())
ax[0].set_yticks(());

m1 = ax[1].imshow(K13_20210513_LiDAR_clipped, cmap = 'viridis', vmin=501, vmax=506)
ax[1].set_title('KID-13 2021-05-13\nLiDAR')
fig.colorbar(m1, label='Elevation (m)', extend='both', ax=ax[1])
ax[1].set_facecolor('k')
ax[1].set_xticks(())
ax[1].set_yticks(());

diff = K13_20210513_LiDAR_reproj-K13_20201109_LiDAR_clipped
m2 = ax[2].imshow(diff, cmap = 'RdBu', vmin=-0.05, vmax=0.05)
ax[2].set_title('KID-13\n2020-11-09 to 2021-05-13\nLiDAR')
fig.colorbar(m2, label='Elevation (m)', extend='both', ax=ax[2])
ax[2].set_facecolor('k')
ax[2].set_xticks(())
ax[2].set_yticks(());

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(8,4))

m0 = ax[0].imshow(K13_20210604_LiDAR_clipped, cmap = 'viridis', vmin=501, vmax=506)
ax[0].set_title('KID-13 2021-06-04\nLiDAR')
fig.colorbar(m0, label='Elevation (m)', extend='both', ax=ax[0])
ax[0].set_facecolor('k')
ax[0].set_xticks(())
ax[0].set_yticks(());

m1 = ax[1].imshow(K13_20210513_LiDAR_clipped, cmap = 'viridis', vmin=501, vmax=506)
ax[1].set_title('KID-13 2021-09-13\nLiDAR')
fig.colorbar(m1, label='Elevation (m)', extend='both', ax=ax[1])
ax[1].set_facecolor('k')
ax[1].set_xticks(())
ax[1].set_yticks(());

diff = K13_20210913_LiDAR_reproj-K13_20210604_LiDAR_clipped
m2 = ax[2].imshow(diff, cmap = 'RdBu', vmin=-0.05, vmax=0.05)
ax[2].set_title('KID-13\n2021-06-04 to 2021-09-13\nLiDAR')
fig.colorbar(m2, label='Elevation (m)', extend='both', ax=ax[2])
ax[2].set_facecolor('k')
ax[2].set_xticks(())
ax[2].set_yticks(());

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(15,4))

m0 = ax[0].imshow(K13_20201109_sfm_clipped, cmap = 'viridis', vmin=501, vmax=506)
ax[0].set_title('KID-13 2020-11-09\nSfM')
fig.colorbar(m0, label='Elevation (m)', extend='both', ax=ax[0])
ax[0].set_facecolor('k')
ax[0].set_xticks(())
ax[0].set_yticks(());

m1 = ax[1].imshow(K13_20210208_sfm_reproj, cmap = 'viridis', vmin=501, vmax=506)
ax[1].set_title('KID-13 2021-02-08\nSfM')
fig.colorbar(m1, label='Elevation (m)', extend='both', ax=ax[1])
ax[1].set_facecolor('k')
ax[1].set_xticks(())
ax[1].set_yticks(());

m2 = ax[2].imshow(K13_20210406_sfm_reproj, cmap = 'viridis', vmin=501, vmax=506)
ax[2].set_title('KID-13 2021-04-06\nSfM')
fig.colorbar(m2, label='Elevation (m)', extend='both', ax=ax[2])
ax[2].set_facecolor('k')
ax[2].set_xticks(())
ax[2].set_yticks(());

m3 = ax[3].imshow(K13_20210513_sfm_reproj, cmap = 'viridis', vmin=501, vmax=506)
ax[3].set_title('KID-13 2021-05-13\nSfM')
fig.colorbar(m3, label='Elevation (m)', extend='both', ax=ax[3])
ax[3].set_facecolor('k')
ax[3].set_xticks(())
ax[3].set_yticks(());

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15,4))

m0 = ax[0].imshow(K13_20210208_sfm_reproj-K13_20201109_sfm_clipped, cmap = 'RdBu', vmin=-0.05, vmax=0.05)
ax[0].set_title('KID-13\n2020-11-09 to 2021-02-08\nSfM')
fig.colorbar(m0, label='Elevation (m)', extend='both', ax=ax[0])
ax[0].set_facecolor('k')
ax[0].set_xticks(())
ax[0].set_yticks(());

m1 = ax[1].imshow(K13_20210406_sfm_reproj-K13_20201109_sfm_clipped, cmap = 'RdBu', vmin=-0.05, vmax=0.05)
ax[1].set_title('KID-13\n2020-11-09 to 2021-04-06\nSfM')
fig.colorbar(m1, label='Elevation (m)', extend='both', ax=ax[1])
ax[1].set_facecolor('k')
ax[1].set_xticks(())
ax[1].set_yticks(());

m2 = ax[2].imshow(K13_20210513_sfm_reproj-K13_20201109_sfm_clipped, cmap = 'RdBu', vmin=-0.05, vmax=0.05)
ax[2].set_title('KID-13\n2020-11-09 to 2021-05-13\nSfM')
fig.colorbar(m2, label='Elevation (m)', extend='both', ax=ax[2])
ax[2].set_facecolor('k')
ax[2].set_xticks(())
ax[2].set_yticks(());